In [1]:
import math
import pandas as pd 
import numpy as np


data = pd.read_csv('cat3.csv')


# print(data.head())


#features
x = data.drop(['index', 'galex_objid', 'sdss_objid', 'class', 'pred'], axis = 1) 
# print(x.head())


#class
y = data['class']
# print(y.head())

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))



#fix scientifci notations to normal floats
# def scitofloat(x):
#     return x

In [3]:
#normalise the values of x

outercnt = 0
for index, row in x.iterrows():
    innercnt = 0
    for column in x:
#         x.at[outercnt, column] = scitofloat(x.iloc[outercnt, innercnt])
        x.at[outercnt, column] = sigmoid(x.iloc[outercnt, innercnt])
#         print(outercnt, column, '\t\t', x.iloc[outercnt, innercnt])
        innercnt += 1
    outercnt += 1    

#     print('\n\n')

print(x.head())

     u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
0  1.0  1.0  1.0  1.0  1.0      0.520923      0.516307      0.511283   
1  1.0  1.0  1.0  1.0  1.0      0.525265      0.519693      0.513627   
2  1.0  1.0  1.0  1.0  1.0      0.520914      0.516300      0.511278   
3  1.0  1.0  1.0  1.0  1.0      0.517817      0.513885      0.509607   
4  1.0  1.0  1.0  1.0  1.0      0.514838      0.511563      0.508000   

   extinction_i  extinction_z  ...       g-z       r-i       r-z       i-z  \
0      0.508385      0.506237  ...  0.384160  0.450753  0.425372  0.474241   
1      0.510127      0.507533  ...  0.612969  0.601924  0.597798  0.495702   
2      0.508381      0.506235  ...  0.502978  0.467984  0.511389  0.543341   
3      0.507139      0.505310  ...  0.965257  0.766892  0.889908  0.710736   
4      0.505945      0.504422  ...  0.910999  0.621933  0.709023  0.596976   

    fuv-nuv     fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  
0  0.290620  0.154052  0.150864  0

In [4]:
def info_dataset(data, verbose=True):
    label1, label2 = 0, 0
    data_size = len(data)
    for index, row in data.iterrows():
        label = row['class']
#         print(label)
        if label == 1:
            label1 += 1
        else:
            label2 += 1
    if verbose:
        print('Total of samples: %d' % data_size)
        print('Total label 1: %d' % label1)
        print('Total label 0: %d' % label2)
    return [len(data), label1, label2]

In [5]:
p = 0.6
_, label1, label2 = info_dataset(data,False)

info_dataset(data)

Total of samples: 4295
Total label 1: 3824
Total label 0: 471


[4295, 3824, 471]

In [6]:
#get training set and test set

from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 15)

train_set = pd.concat([xTrain, yTrain], axis=1, sort=False)
test_set = pd.concat([xTest, yTest], axis=1, sort=False)


print(train_set.iloc[0:4])
# print(train_set.iloc[0][1])

        u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
3350  1.0  1.0  1.0  1.0  1.0      0.522790      0.517763      0.512291   
2195  1.0  1.0  1.0  1.0  1.0      0.598732      0.577330      0.553721   
4274  1.0  1.0  1.0  1.0  1.0      0.513876      0.510813      0.507481   
4256  1.0  1.0  1.0  1.0  1.0      0.517623      0.513734      0.509502   

      extinction_i  extinction_z  ...       r-i       r-z       i-z   fuv-nuv  \
3350      0.509134      0.506794  ...  0.477962  0.546235  0.567996  0.341868   
2195      0.539989      0.529773  ...  0.525673  0.572446  0.547123  0.201572   
4274      0.505559      0.504135  ...  0.525156  0.440654  0.415999  0.293123   
4256      0.507062      0.505253  ...  0.556995  0.579544  0.522965  0.247259   

         fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  class  
3350  0.088467  0.057180  0.046900  0.051004  0.039272      1  
2195  0.145669  0.100441  0.090785  0.082650  0.069401      1  
4274  0.093665  0.094642  

In [7]:
def euclidian_dist(p1, p2):
    dim, sum_ = len(p1), 0
    for index in range(dim - 1):
        sum_ += math.pow(p1[index] - p2[index], 2)
    return math.sqrt(sum_)

In [8]:
def knn(train_set, new_sample, K):

#     print('train set', train_set, type(train_set))
#     print('new sample', new_sample, type(new_sample))
    dists, train_size = {}, len(train_set)

    for i in range(train_size):
#         print(train_set.iloc[i-1].values.tolist())
#         print(new_sample.values.tolist()[0])
#         print('\n\n')
        d = euclidian_dist(train_set.iloc[i-1].values.tolist(), new_sample.values.tolist()[0])
        dists[i] = d
        
#     print('done')
    
    k_neighbors = sorted(dists, key=dists.get)[:K]
    
    qty_label1, qty_label2 = 0, 0
    
    for index in k_neighbors:
#         print('index, ', index)
#         print(train_set.iloc[index])
        
    
        #fix this
        if train_set.iloc[index].values.tolist()[-1] == 1:
            qty_label1 += 1
        else:
            qty_label2 += 1
        
            
    if qty_label1 > qty_label2:
        return 1
    else:
        return 0

In [9]:
correct, K = 0, 5


count, tp, fp, tn, fn = 0, 0, 0, 0, 0


for index, sample in test_set.iterrows():
    print(count)
    count +=1
    sample = pd.DataFrame(sample)
    sample = sample.T

    label = knn(train_set, sample, K)

    if label == sample.iloc[0].values.tolist()[-1]:
        correct += 1
    
    print('predicted, actual', label, sample.iloc[0].values.tolist()[-1])
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 1:
        tp += 1
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 0:
        fp += 1 
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 1:
        fn += 1    
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 0:
        tn += 1    

print('total: ', count)
print('correct: ', correct)
print('\n\n')

matrix_list = np.array([[tp, fp], [fn, tn]])

# matrix = pd.DataFrame({'Actual +ve': matrix_list[:, 0], 'Actual -ve': matrix_list[:, 1]})
matrix = pd.DataFrame(matrix_list, index=['Pred +ve', 'Pred -ve'], columns=['Actual +ve', 'Actual -ve'])



print(matrix)

0
predicted, actual 1 0.0
1
predicted, actual 1 1.0
2
predicted, actual 1 1.0
3
predicted, actual 1 1.0
4
predicted, actual 1 1.0
5
predicted, actual 1 1.0
6
predicted, actual 1 1.0
7
predicted, actual 1 1.0
8
predicted, actual 1 1.0
9
predicted, actual 1 1.0
10
predicted, actual 1 1.0
11
predicted, actual 1 1.0
12
predicted, actual 1 1.0
13
predicted, actual 1 0.0
14
predicted, actual 1 1.0
15
predicted, actual 1 1.0
16
predicted, actual 1 0.0
17
predicted, actual 1 1.0
18
predicted, actual 1 1.0
19
predicted, actual 1 1.0
20
predicted, actual 1 1.0
21
predicted, actual 1 1.0
22
predicted, actual 1 1.0
23
predicted, actual 1 1.0
24
predicted, actual 0 1.0
25
predicted, actual 1 1.0
26
predicted, actual 1 1.0
27
predicted, actual 1 1.0
28
predicted, actual 1 0.0
29
predicted, actual 1 1.0
30
predicted, actual 1 0.0
31
predicted, actual 1 1.0
32
predicted, actual 1 1.0
33
predicted, actual 1 0.0
34
predicted, actual 1 1.0
35
predicted, actual 1 1.0
36
predicted, actual 1 1.0
37
predicte

predicted, actual 1 1.0
298
predicted, actual 1 1.0
299
predicted, actual 1 1.0
300
predicted, actual 1 1.0
301
predicted, actual 1 1.0
302
predicted, actual 1 1.0
303
predicted, actual 1 1.0
304
predicted, actual 1 1.0
305
predicted, actual 1 1.0
306
predicted, actual 1 1.0
307
predicted, actual 1 1.0
308
predicted, actual 1 1.0
309
predicted, actual 1 1.0
310
predicted, actual 1 1.0
311
predicted, actual 1 1.0
312
predicted, actual 1 1.0
313
predicted, actual 1 1.0
314
predicted, actual 1 1.0
315
predicted, actual 1 1.0
316
predicted, actual 1 1.0
317
predicted, actual 1 1.0
318
predicted, actual 1 1.0
319
predicted, actual 1 1.0
320
predicted, actual 1 1.0
321
predicted, actual 1 0.0
322
predicted, actual 1 1.0
323
predicted, actual 1 1.0
324
predicted, actual 1 1.0
325
predicted, actual 1 1.0
326
predicted, actual 1 1.0
327
predicted, actual 1 1.0
328
predicted, actual 1 1.0
329
predicted, actual 1 1.0
330
predicted, actual 1 0.0
331
predicted, actual 1 1.0
332
predicted, actual 1 

predicted, actual 1 1.0
591
predicted, actual 1 1.0
592
predicted, actual 1 1.0
593
predicted, actual 1 1.0
594
predicted, actual 1 1.0
595
predicted, actual 1 1.0
596
predicted, actual 1 1.0
597
predicted, actual 1 1.0
598
predicted, actual 1 1.0
599
predicted, actual 1 1.0
600
predicted, actual 1 1.0
601
predicted, actual 1 0.0
602
predicted, actual 1 1.0
603
predicted, actual 1 1.0
604
predicted, actual 0 1.0
605
predicted, actual 1 1.0
606
predicted, actual 1 1.0
607
predicted, actual 1 1.0
608
predicted, actual 1 1.0
609
predicted, actual 1 1.0
610
predicted, actual 1 1.0
611
predicted, actual 1 1.0
612
predicted, actual 1 1.0
613
predicted, actual 1 1.0
614
predicted, actual 1 1.0
615
predicted, actual 1 1.0
616
predicted, actual 1 1.0
617
predicted, actual 1 1.0
618
predicted, actual 1 1.0
619
predicted, actual 1 1.0
620
predicted, actual 1 1.0
621
predicted, actual 1 0.0
622
predicted, actual 1 0.0
623
predicted, actual 1 1.0
624
predicted, actual 1 1.0
625
predicted, actual 1 

In [10]:
print("Train set size: %d" % len(train_set))
print("Test set size: %d" % len(test_set))
print("Correct predicitons: %d" % correct)
print("Accuracy: %.2f%%" % (100 * correct / len(test_set)))

precision = tp/(tp + fp)
recall = tp/(tp + fn)
fscore = 2*((precision * recall)/(precision + recall))

print('precision: ', precision)
print('recall: ', recall)
print('fscore: ', fscore)

Train set size: 3436
Test set size: 859
Correct predicitons: 763
Accuracy: 88.82%
precision:  0.8965922444183314
recall:  0.9896238651102465
fscore:  0.9408138101109742
